# old results

## DRMM result相关的训练数据

In [1]:
import json
import pickle
import numpy as np
import pandas as pd
from pprint import pprint

In [21]:
result = pd.read_table('./drmm_results/result_grissom_rerank.txt',sep='\t',header=None)
pprint(type(result))
pprint(result.head)
pprint(result[0].unique().shape)

<class 'pandas.core.frame.DataFrame'>
<bound method NDFrame.head of           0  1                 2    3         4     5
0       301  0       FBIS4-46734    0  6.859538  drmm
1       301  0       FBIS4-39330    1  6.754804  drmm
2       301  0        FBIS4-7811    2  6.686247  drmm
3       301  0       FBIS3-24145    3  6.658622  drmm
4       301  0       FBIS3-23986    4  6.445690  drmm
...     ... ..               ...  ...       ...   ...
241388  700  0       FBIS3-31959  995  2.720761  drmm
241389  700  0  FR940921-0-00153  996  2.718753  drmm
241390  700  0     LA110490-0093  997  2.718751  drmm
241391  700  0     LA122889-0096  998  2.717326  drmm
241392  700  0     LA102690-0197  999  2.715926  drmm

[241393 rows x 6 columns]>
(249,)


In [22]:
pprint(result[2].unique().shape)  # 共有158790篇不同的文档

(158790,)


In [3]:
with open('./robust04_data/docset_fbis5.pkl', 'rb') as f: docset_fbis5 = pickle.load(f)
with open('./robust04_data/docset_fr94.pkl', 'rb') as f: docset_fr94 = pickle.load(f)
with open('./robust04_data/docset_ft.pkl', 'rb') as f: docset_ft = pickle.load(f)
with open('./robust04_data/docset_latimes.pkl', 'rb') as f: docset_latimes = pickle.load(f)

In [4]:
docset = docset_fbis5
docset.update(docset_fr94)
docset.update(docset_ft)
docset.update(docset_latimes)

In [31]:
with open('./docset.pkl', 'wb') as f:
    pickle.dump(docset, f)

In [36]:
len(docset)

528155

In [7]:
count = 0
for item in list_1: 
    if item not in set_2: count += 1
count

0

In [13]:
length = []
rank = list(result[3])
count = 0  # 统计索引条目少于300的topic
for i in range(len(rank)-900):
    if rank[i] > rank[i+1]:
        length.append(rank[i]+1)
        if length[-1] < 300:
            count += 1
print(min(length), max(length))
print(count)

6 1000
6


## 制作DRMM数据集

In [32]:
import random
def split_dataset(all_data: dict, train_ratio=0.8, seed=1):
    """
    随机划分数据集
    """
    random.seed(seed)
    train_size = int(len(all_data) * train_ratio)
    train_set, test_set, key_list = {}, {}, list(all_data.keys())
    for i in range(train_size):
        random_index = random.randint(0, len(key_list) - 1)
        train_set[key_list[random_index]] = all_data[key_list[random_index]]
        key_list.pop(random_index)
    for key in key_list: test_set[key] = all_data[key]
    return train_set, test_set

In [23]:
all_data = {}
for index, row in result.iterrows():
    if str(row[0]) not in all_data: all_data[str(row[0])], count = {row[3]: {'doc_id':row[2] , 'score': row[4]}}, 1
    elif count >= 300: continue
    else: 
        all_data[str(row[0])].update({row[3]: {'doc_id':row[2] , 'score': row[4]}})
        count += 1

In [24]:
for key in list(all_data.keys()):
    if len(all_data[key]) < 300: del all_data[key]
len(all_data)

243

In [30]:
# 统计一个相关文档都没有的ranked list
doc0count = 0
for key in list(all_data.keys()):
    count = 0
    for index in all_data[key]:
        if all_data[key][index]['doc_id'] in gt[key]:
            count += 1
            break
    if count == 0: 
        doc0count += 1
        del all_data[key]
print(doc0count)
print(len(all_data))

4
239


In [36]:
import os
# 生成5-fold数据集
for i in range(1, 6):
    train_set, test_set = split_dataset(all_data, seed=i)
    if not os.path.exists('./drmm_results_clean/split_{}/'.format(i)): os.makedirs('./drmm_results_clean/split_{}/'.format(i))
    with open('./drmm_results_clean/split_{}/drmm_train_s{}.pkl'.format(i, i), 'wb') as f: 
        pickle.dump(train_set, f)
    with open('./drmm_results_clean/split_{}/drmm_test_s{}.pkl'.format(i, i), 'wb') as f: 
        pickle.dump(test_set, f)
with open('./drmm_results_clean/DRMM_all.pkl', 'wb') as f: pickle.dump(all_data, f)

### DRMM结果统计
* 可见模型结果一共包含249个topic的结果（少了一个，但是问题不大）
* 结果条目数最少是6，最大是1000
* 共有243个结果条目数大于等于300的topic，可以用作序列截断任务

# 文档相关的输入数据

In [1]:
import pickle
from pprint import pprint

In [2]:
# 提取topic与对应的相关文档的数据字典
with open('robust04_data/split_1/rob04_bm25_top1000.train.s1.pkl', 'rb') as f:
    train_s1 = pickle.load(f)
with open('robust04_data/split_1/rob04_bm25_top1000.dev.s1.pkl', 'rb') as f:
    dev_s1 = pickle.load(f)
with open('robust04_data/split_1/rob04_bm25_top1000.test.s1.pkl', 'rb') as f:
    test_s1 = pickle.load(f)

# 提取原文档的描述信息字典
with open('robust04_data/split_1/rob04_bm25_docset_top1000.train.s1.pkl', 'rb') as f:
    train_s1_doc = pickle.load(f)
with open('robust04_data/split_1/rob04_bm25_docset_top1000.dev.s1.pkl', 'rb') as f:
    dev_s1_doc = pickle.load(f)
with open('robust04_data/split_1/rob04_bm25_docset_top1000.test.s1.pkl', 'rb') as f:
    test_s1_doc = pickle.load(f)

In [5]:
pprint(train_s1['queries'][0].keys())
pprint(list(train_s1_doc.keys())[:10])  # 展示前10个文档标号
print(train_s1_doc['FR940527-1-00246'].keys(), len(train_s1_doc))  # 展示每一个文档的描述信息键值

dict_keys(['query_id', 'query_text', 'relevant_documents', 'num_rel', 'retrieved_documents', 'num_ret', 'num_rel_ret'])
['FR940527-1-00246',
 'FT924-3705',
 'FT923-13842',
 'FT924-13624',
 'LA021190-0103',
 'FT931-12873',
 'FT941-9456',
 'FR941221-2-00097',
 'FR940119-1-00020',
 'LA120190-0121']
dict_keys(['title', 'abstractText']) 115502


In [6]:
print(train_s1_doc['FR941221-2-00097']['abstractText'])

   SGA/DAA 94&hyph;24, Local Learning Laboratory   Budget Detail Sheet       Applicant:       &blank;   1Category   2&blank;   1Year 1   2Federal funds requested   2Non-Federal funds      Service Delivery Budget:    Reflects planned improvements in access and method of delivery of services to benefit the customer.  &blank;  &blank;    Program/Service Integration Budget:    Reflects additional programs/services to be integrated into the local One-Stop system.  &blank;  &blank;     Hardware Budget:    Reflects hardware acquisitions for programmatic, direct customer service, and labor market information purposes.  &blank;  &blank;     Software Budget:    Reflects software enhancements to improve case management and/or delivery of services (including conveyance of labor market information).  &blank;  &blank;     Space and Premises Budget:    Reflects reconfiguration and upgrading of space to create an office environment more ``user friendly'' and attractive to customers.  &blank;  &blank; 

## 生成文档相关的统计数据和表征

In [16]:
import re
import json
from collections import defaultdict
from pprint import pprint

In [17]:
test_text = 'SGA/DAA 94&hyph;24, Local Learning Laboratory   Budget Detail Sheet       Applicant:       &blank;   1Category   2&blank;   1Year 1   2Federal funds requested   2Non-Federal funds      Service Delivery Budget:    Reflects planned improvements in access and method of delivery of services to benefit the customer.  &blank;  &blank;    Program/Service Integration Budget:    Reflects additional programs/services to be integrated into the local One-Stop system.  &blank;  &blank;     Hardware Budget:    Reflects hardware acquisitions for programmatic, direct customer service, and labor market information purposes.  &blank;  &blank;     Software Budget:    Reflects software enhancements to improve case management and/or delivery of services (including conveyance of labor market information).  &blank;  &blank;     Space and Premises Budget:    Reflects reconfiguration and upgrading of space to create an office environment more ``user friendly'' and attractive to customers.  &blank;  &blank;      Testing and Evaluation Budget:    Reflects testing and evaluating innovations in One-Stop system delivery.  &blank;  &blank;     Marketing/Dissemination Budget:    Reflects staff and material costs for marketing/dissemination activities.  &blank;  &blank;     Training Budget:    Specialized staff training to achieve system delivery, integration, technology (hardware and software), testing and evaluation, marketing/dissemination activities.  &blank;  &blank;  n,s    Total  &blank;  &blank;            [FR Doc. 94&hyph;31294 Filed 12&hyph;20&hyph;94; 8:45 am]       BILLING CODE 4510&hyph;30&hyph;M'
# test_text = train_s1_doc['FR941221-2-00097']['abstractText']
clean = lambda t: re.sub('[,?;*!%^&_+():\[\]{}`~@#$=+\\|/<>.\'\"\d]', ' ', t.replace('"', ' ').replace('/', ' ').replace('\\', ' ').replace("'", ' ').replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('-', ' ').replace('.', '').replace('&hyph;', ' ').replace('&blank;', ' ').strip().lower())
preprocessed_text = clean(test_text)
print(preprocessed_text)

sga daa        local learning laboratory   budget detail sheet       applicant             category         year      federal funds requested    non federal funds      service delivery budget     reflects planned improvements in access and method of delivery of services to benefit the customer          program service integration budget     reflects additional programs services to be integrated into the local one stop system           hardware budget     reflects hardware acquisitions for programmatic  direct customer service  and labor market information purposes           software budget     reflects software enhancements to improve case management and or delivery of services  including conveyance of labor market information            space and premises budget     reflects reconfiguration and upgrading of space to create an office environment more   user friendly and attractive to customers            testing and evaluation budget     reflects testing and evaluating innovations in o

In [18]:
stoplist = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
test_list = [word for word in preprocessed_text.lower().split() if word not in stoplist]

# remove words that appear only once
frequency = defaultdict(int)
for token in test_list:
    frequency[token] += 1

texts = [token for token in test_list if frequency[token] > 1]

In [20]:
print(texts)

['local', 'budget', 'federal', 'funds', 'federal', 'funds', 'service', 'delivery', 'budget', 'reflects', 'delivery', 'services', 'customer', 'service', 'integration', 'budget', 'reflects', 'services', 'local', 'one', 'stop', 'system', 'hardware', 'budget', 'reflects', 'hardware', 'customer', 'service', 'labor', 'market', 'information', 'software', 'budget', 'reflects', 'software', 'delivery', 'services', 'labor', 'market', 'information', 'space', 'budget', 'reflects', 'space', 'testing', 'evaluation', 'budget', 'reflects', 'testing', 'one', 'stop', 'system', 'delivery', 'marketing', 'dissemination', 'budget', 'reflects', 'staff', 'marketing', 'dissemination', 'activities', 'training', 'budget', 'staff', 'training', 'system', 'delivery', 'integration', 'hardware', 'software', 'testing', 'evaluation', 'marketing', 'dissemination', 'activities']


## 将所有文档清洗转换为token列表之后，可以使用gensim操作

In [28]:
# 文本清洗函数
def clean_texts(texts, stoplist):
    clean = lambda t: re.sub('[,?;*!%^&_+():\[\]{}`~@#$=+\\|/<>.\'\"\d]', ' ', t.replace('"', ' ').replace('/', ' ').replace('\\', ' ').replace("'", ' ').replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('-', ' ').replace('.', '').replace('&hyph;', ' ').replace('&blank;', ' ').strip().lower())
    preprocessed_text = clean(texts)
    text_list = [word for word in preprocessed_text.lower().split() if word not in stoplist]
    # remove words that appear only once
    frequency = defaultdict(int)
    for token in text_list:
        frequency[token] += 1
    clean_texts = [token for token in frequency if frequency[token] > 1]
    return clean_texts

In [29]:
docset['FT934-3024']

{'title': 'FT  10 DEC 93 / UK Company News: Standard Chartered in Pounds 100m\npreference issue',
 'abstractText': "Standard Chartered, the UK-based international banking group, yesterday\nstrengthened its capital base with a Pounds 100m issue of non-cumulative\npreference shares, writes John Gapper. The issue raises its core tier 1\nratio of capital to risk-weighted assets.\nStandard's share price has risen sharply recently, driven by its exposure to\nemerging markets in the Asia Pacific region. But there has been speculation\nover whether it would be forced into a rights issue to strengthen capital.\nThe bank is also exposed to the appreciation of the dollar against sterling\nbecause its capital is sterling-denominated, while most of its assets are in\ncurrencies which are more closely linked to the dollar.\nIt has had a lower ratio of tier 1 capital - comprising equity, risk capital\nincluding non-cumulative preference shares, and retained earnings - to its\nrisk-weighted assets tha

In [32]:
# 根据docset生成corpus
corpus = []
for key in docset:
    texts = docset[key]['title'] + docset[key]['abstractText']
    document = clean_texts(texts, stoplist)
    corpus.append(document)
len(corpus)

528155

In [35]:
# 将结果保存到pkl文件中
with open('./corpus.pkl', 'wb') as f:
    pickle.dump(corpus, f)

## 生成BM25结果

In [2]:
with open('./robust04_data/split_1/rob04_bm25_top1000.train.s1.pkl', 'rb') as f: bm25_train = pickle.load(f)
with open('./robust04_data/split_1/rob04_bm25_top1000.dev.s1.pkl', 'rb') as f: bm25_dev = pickle.load(f)
with open('./robust04_data/split_1/rob04_bm25_top1000.test.s1.pkl', 'rb') as f: bm25_test = pickle.load(f)

In [3]:
for i in range(0, 10): 
    print(bm25_test['queries'][0]['retrieved_documents'][i]['is_relevant'])

True
True
False
True
False
True
True
True
True
False


In [4]:
type(bm25_train['queries'])

list

In [5]:
bm25 = bm25_train['queries'] + bm25_dev['queries'] + bm25_test['queries']

In [6]:
bm25[0].keys()

dict_keys(['query_id', 'query_text', 'relevant_documents', 'num_rel', 'retrieved_documents', 'num_ret', 'num_rel_ret'])

In [7]:
aux_set = set()
for i in range(250): aux_set.add(bm25[i]['query_id'])
len(aux_set)

250

In [10]:
bm25[0]['relevant_documents'][0]

'FBIS3-10082'

In [11]:
bm25[0]['retrieved_documents'][2]

{'doc_id': 'FBIS4-41991',
 'rank': 3,
 'bm25_score': 5.63055377,
 'norm_bm25_score': 3.1348520653230927,
 'is_relevant': False}

In [12]:
bm25[0]['num_ret']

1000

In [38]:
bm25_dataset = {}
for item in bm25: 
    if len(item['retrieved_documents']) >=300: bm25_dataset[item['query_id']] = {'query_text': item['query_text'], 'retrieved_documents': item['retrieved_documents'][:300]}

In [39]:
# 统计一个相关文档都没有的ranked list
doc0count = 0
for key in list(bm25_dataset.keys()):
    count = 0
    for ret_doc in bm25_dataset[key]['retrieved_documents']:
        # if ret_doc['doc_id'] in gt[key]:
        if ret_doc['is_relevant']: 
            count += 1
            break
    if count == 0: 
        doc0count += 1
        del bm25_dataset[key]
print(doc0count)
print(len(bm25_dataset))

2
241


In [40]:
import os
# 生成5-fold数据集
for i in range(1, 6):
    train_set, test_set = split_dataset(bm25_dataset, seed=i)
    if not os.path.exists('./BM25_results_clean/split_{}/'.format(i)): os.makedirs('./BM25_results_clean/split_{}/'.format(i))
    with open('./BM25_results_clean/split_{}/BM25_train_s{}.pkl'.format(i, i), 'wb') as f: 
        pickle.dump(train_set, f)
    with open('./BM25_results_clean/split_{}/BM25_test_s{}.pkl'.format(i, i), 'wb') as f: 
        pickle.dump(test_set, f)
with open('./BM25_results_clean/BM25_all.pkl', 'wb') as f:
    pickle.dump(bm25_dataset, f)

In [35]:
with open('./BM25_results/split_1/BM25_train_s1.pkl', 'rb') as f: tr1 = pickle.load(f)
with open('./BM25_results/split_2/BM25_train_s2.pkl', 'rb') as f: tr2 = pickle.load(f)

In [36]:
tr1.keys()

dict_keys(['368', '307', '429', '339', '305', '338', '414', '365', '669', '318', '654', '664', '611', '625', '419', '400', '357', '679', '325', '647', '636', '655', '436', '424', '432', '301', '304', '666', '433', '347', '657', '411', '443', '363', '607', '328', '321', '331', '696', '379', '310', '646', '389', '416', '675', '622', '334', '407', '423', '689', '352', '630', '434', '641', '435', '610', '426', '337', '614', '332', '690', '674', '450', '375', '408', '449', '627', '384', '649', '632', '329', '651', '412', '637', '624', '425', '662', '673', '700', '616', '341', '639', '602', '342', '371', '406', '687', '439', '694', '373', '618', '326', '383', '405', '420', '377', '319', '324', '336', '309', '353', '660', '422', '421', '427', '633', '613', '315', '447', '629', '672', '698', '346', '386', '608', '653', '320', '659', '685', '695', '626', '303', '446', '308', '621', '370', '684', '676', '648', '391', '367', '665', '330', '682', '392', '620', '350', '656', '448', '343', '314', '3

In [37]:
tr2.keys()

dict_keys(['439', '425', '335', '355', '353', '613', '429', '387', '696', '413', '619', '623', '441', '415', '379', '402', '397', '328', '352', '673', '384', '655', '630', '641', '622', '324', '690', '634', '340', '665', '693', '433', '331', '325', '636', '685', '609', '645', '667', '377', '393', '605', '398', '423', '421', '321', '400', '629', '399', '381', '428', '431', '654', '601', '608', '676', '406', '319', '687', '651', '659', '326', '653', '660', '343', '395', '688', '417', '657', '699', '445', '640', '614', '672', '677', '304', '666', '436', '618', '664', '449', '420', '611', '683', '434', '646', '401', '612', '308', '643', '638', '370', '652', '424', '389', '371', '648', '427', '361', '616', '307', '358', '682', '306', '416', '373', '351', '344', '637', '603', '376', '394', '649', '625', '447', '357', '606', '621', '632', '302', '315', '334', '440', '442', '374', '404', '661', '313', '350', '363', '341', '327', '336', '391', '310', '631', '426', '380', '698', '390', '633', '4

In [16]:
gt = {}
for item in bm25: gt[item['query_id']] = item['relevant_documents']

In [18]:
with open('./robust04_gt.pkl', 'wb') as f:
    pickle.dump(gt, f)

In [42]:
with open('./drmm_results/DRMM_all.pkl', 'rb') as f: drmm_all = pickle.load(f)
with open('./BM25_results/BM25_all.pkl', 'rb') as f: bm25_all = pickle.load(f)

In [46]:
drmm_ranked_list = {}
for key in drmm_all: drmm_ranked_list[key] = [drmm_all[key][index]['doc_id'] for index in drmm_all[key]]
drmm_ranked_list['301'][:5]

['FBIS4-46734', 'FBIS4-39330', 'FBIS4-7811', 'FBIS3-24145', 'FBIS3-23986']

In [49]:
bm25_ranked_list = {}
for key in bm25_all: bm25_ranked_list[key] = [bm25_all[key]['retrieved_documents'][i]['doc_id'] for i in range(300)]
bm25_ranked_list['301'][:5]

['FBIS3-21961', 'LA112489-0141', 'FBIS4-41991', 'LA121990-0141', 'FBIS4-19535']

In [50]:
with open('./drmm_ranked_list.json', 'w') as f: json.dump(drmm_ranked_list, f)
with open('./bm25_ranked_list.json', 'w') as f: json.dump(bm25_ranked_list, f) 

## 针对matchzoo的结果制作ranked list

In [2]:
with open('information retrieval/robust04/matchzoo/results/drmm_tks.pkl', 'rb') as f: tks_all = pickle.load(f)
tks_all.head()

,id_left,id_right,label,relation_score
0,434,FBIS4-62284,1.0,5.472672
1,627,FBIS4-17334,0.0,-1.677305
2,408,LA090889-0108,0.0,-9.801664
3,347,FR940810-0-00217,0.0,-4.566301
4,663,FT941-9723,0.0,-4.963085


In [11]:
tks_all.sort_values(by=['id_left'], inplace=True)

In [12]:
tks_all.head()

,id_left,id_right,label,relation_score
175761,301,FBIS3-37399,0.0,-9.958324
136069,301,FBIS3-10513,0.0,-7.961651
147589,301,LA121290-0151,0.0,-7.406104
164235,301,FT934-2699,0.0,-9.071196
88028,301,FBIS3-13584,0.0,-8.790793


In [13]:
len(tks_all[tks_all['id_left'] == '301'])

1364

In [19]:
tks_all[tks_all['id_left'] == '301'].sort_values(by=['relation_score'], ascending=False).head()

,id_left,id_right,label,relation_score
166166,301,FBIS3-41131,1.0,6.436135
104024,301,FBIS3-41212,1.0,6.268208
109656,301,FBIS3-41349,1.0,6.176438
27215,301,FBIS4-7811,1.0,6.106179
71371,301,FBIS3-41293,1.0,5.937361


In [20]:
queries = list(pd.unique(tks_all['id_left']))
queries[:3]

['301', '302', '303']

In [37]:
tks_ranked_list = {}
for query in queries:
    tks_ranked_list[query] = []
    qerls_df = tks_all[tks_all['id_left'] == query].sort_values(by=['relation_score'], ascending=False)
    for row in qerls_df.iterrows():
        tks_ranked_list[query].append({'doc_id': row[1]['id_right'], 'score': row[1]['relation_score']})

In [42]:
all_data = {}
for key in tks_ranked_list:
    if len(tks_ranked_list[key]) >= 300: all_data[key] = tks_ranked_list[key][:300]
len(all_data)

243

In [43]:
import random
def split_dataset(all_data: dict, train_ratio=0.8, seed=1):
    """
    随机划分数据集
    """
    random.seed(seed)
    train_size = int(len(all_data) * train_ratio)
    train_set, test_set, key_list = {}, {}, list(all_data.keys())
    for i in range(train_size):
        random_index = random.randint(0, len(key_list) - 1)
        train_set[key_list[random_index]] = all_data[key_list[random_index]]
        key_list.pop(random_index)
    for key in key_list: test_set[key] = all_data[key]
    return train_set, test_set

In [44]:
import os
# 生成5-fold数据集
for i in range(1, 6):
    train_set, test_set = split_dataset(all_data, seed=i)
    if not os.path.exists('./ranked list truncation/data_prep/my_results/drmm_tks_results/split_{}/'.format(i)): os.makedirs('./ranked list truncation/data_prep/my_results/drmm_tks_results/split_{}/'.format(i))
    with open('./ranked list truncation/data_prep/my_results/drmm_tks_results/split_{}/drmm_tks_train_s{}.pkl'.format(i, i), 'wb') as f: pickle.dump(train_set, f)
    with open('./ranked list truncation/data_prep/my_results/drmm_tks_results/split_{}/drmm__tks_test_s{}.pkl'.format(i, i), 'wb') as f: pickle.dump(test_set, f)
with open('./ranked list truncation/data_prep/my_results/drmm_tks_results/DRMM_TKS_all.pkl', 'wb') as f: pickle.dump(all_data, f)